In [1]:
from transformers import AutoTokenizer

model_name = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/opt/conda/envs/python311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from transformers import AutoModelForCausalLM,BitsAndBytesConfig

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}



# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

new_model = "llama-2-7b-clinical-trials"

model = AutoModelForCausalLM.from_pretrained(
    f'./{new_model}',
    quantization_config=bnb_config,
    device_map=device_map
)

Loading checkpoint shards: 100% 2/2 [00:06<00:00,  3.37s/it]
/opt/conda/envs/python311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/python311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  

In [3]:
from transformers import pipeline
prompt = "I am 35 year old man with diabetes. What clinical trial can I participate?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result)

[{'generated_text': 'Congenital Adrenal Hyperplasia: Calcium Channels as Therapeutic Targets. J Clin Endocrinol Metab. 2003;88(12):5309-5316.\n11. Hershman R, et al. The effects of corticotropin-releasing factor on the adrenal cortex. J Clin Endocrinol Metab. 1977;45(3):477-483.\n12. Hershman R, et al. The effects of corticotropin-releasing factor on the adrenal cortex. J Clin Endocrinol Metab. 1977;45(3):477-483.\n13. Hershman R, et al. The effects of corticotropin'}]
